In [6]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import datetime
import pandas as pd
import numpy as np
import os
os.chdir('C:\\Users\\TAN\\Documents\\IM\\RealEstateProject\\tmp')


def algandmed(x):
    path="C:\\Users\\TAN\\Documents\\IM\\RealEstateProject\\tmp"
    link_file = open(x)
    #path="C:\\Users\\TAN\\Documents\\IM\\RealEstateProject\\tmp"
    for url in link_file.read().splitlines():
        df = dataframe(url)
        dt = str(datetime.date.today())
        df.to_csv(open(path + '_' + dt + '.csv', 'w' , encoding='utf-8'))
        writer = pd.ExcelWriter(path + '_' + dt + '.xlsx')
        df.to_excel(writer, 'Sheet1')
        writer.save()
        print(df.head())
        print(df.tail())
        print(len(df))


def generate_links(x):

    """Parse links from a file ('links.txt')
    and add pagination (+1) to the link"""
    urls = []
    MAX_PAGE_NUM = 1
    MAX_PAGE_DIG = 1
    for i in range(1, MAX_PAGE_NUM + 1):
        page_num = (MAX_PAGE_DIG - len(str(i))) * "0" + str(i)
        url = x + page_num
        urls.append(url)
    print("\n Pages that will be scraped: \n\n", urls)
    return urls


def driver_init():

    """Select the driver and driver options"""
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    # You need to specify the place you have your chrome/firefox etc installed
    options.binary_location = 'C:/Program Files (x86)/Google/Chrome/Application/chrome.exe'
    #options.binary_location = 'C:/Program Files/internet explorer'
    
    driver = webdriver.Chrome("C:/chromedriver_win32/chromedriver.exe")
    #driver = webdriver.Chrome(options=options)
    driver.set_window_size(1120, 550)
    return driver
    print('\t Driver initialized \n',
          'Driver: %s \n' % driver,
          'Options: %s' % options)


def get_links(x):

    """Initialize lists"""
    urls = generate_links(x)
    hrefs = []

    """Scrape through list of urls"""
    for idx, url in enumerate(urls, start=1):
        driver = driver_init()
        driver.get(url)
        products = driver.find_elements_by_css_selector('.object-title-a')

        """For each url scrape all items into a list"""
        num_page_items = len(products)
        print(num_page_items)
        for i in range(num_page_items):
            try:
                href = products[i].get_attribute('href')
                hrefs.append(href)
                """Error handling for IndexError"""
            except IndexError as e:
                print("Error: %s" % e)
            else:
                print('No exceptions occured')
        driver.quit()
        print("\n Page #", idx, url)
        print("+ @href info of producs added to links")

    """Eemaldan linkide nimistust duplikaadid"""
    print(len(hrefs))
    return hrefs


def toote_info(x):

    driver = driver_init()
    links = get_links(x)

    name = []
    rooms = []
    price = []
    pricesqm = []
    numoffloor = []
    saadavus = []
    geoloc =[]
    builtyear=[]
    energymark=[]
    condition=[]
    link = []
    
    #geoloc_elem
    latlong=[]
    for n in links:
        try:
            driver.get(n)
            print(n)
        except Exception as e:
            print("NoSuchElementException \n" + str(e))
            pass
        
        try:
            name.append(driver.find_element_by_css_selector('.large .title').text)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            name.append(None)
        
        try:
            rooms.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[1]/table/tbody/tr[1]/td').text)
            print(rooms)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            rooms.append(None)
        
        try:
            price.append(driver.find_element_by_css_selector('.m-1-2 strong').text)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            price.append(None)
        
        try:
            pricesqm.append(driver.find_element_by_css_selector('.object-m2-price').text)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            pricesqm.append(None)
        
        try:
            numoffloor.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[1]/table/tbody/tr[3]/td').text)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            numoffloor.append(None)    
        
        try:
            if (driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[1]/table/tbody/tr[4]/th').text=='Ehitusaasta'):
                builtyear.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[1]/table/tbody/tr[4]/td').text)
            print(builtyear)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            builtyear.append(None)  

        try:
            #energymark.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[1]/table/tbody/tr[7]/td[contains(text(),'Energiamärgis')]').text)
            print(energymark)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            energymark.append(None)  
        
        try:
            #condition.append(driver.find_element_by_xpath('/html/body/div[2]/div/div[2]/div[3]/div[2]/div/div[1]/table/tbody/tr[5]/td[contains(text(),'Seisukord')]').text)
            print(condition)
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            condition.append(None) 
        
        try:
            geoloc.append(driver.find_element_by_xpath('//*[@id="gmap_img"]/a').get_attribute('href').split('&query=')[1])
        except NoSuchElementException as e:
            print("NoSuchElementException \n" + str(e))
            geoloc.append(None)
        
        try:
            link.append(n)
        except Exception as e:
            print("Exception \n" + str(e))
            link.append(None)

    """Quit the driver to save memory"""
    driver.quit()
    print(name, rooms, price, saadavus)

    """Zip all the lists into a table"""
    data = [{'name': t_name,
             'rooms': t_rooms,
             'price': t_price,
             'pricesqm': t_pricesqm,
             'numoffloor': t_numoffloor,
             'geoloc': t_geoloc,
             'builtyear': t_builtyear,
             'energymark': t_energymark,
             'condition': t_condition,
             'link': t_link} for t_name, t_rooms,
            t_price, t_pricesqm, t_numoffloor, t_geoloc, t_builtyear, t_energymark, t_condition, t_link, in zip(name, rooms, price, pricesqm, numoffloor, geoloc, builtyear, energymark, condition, link)]
    print(data)

    return data


def dataframe(x):
    df = pd.DataFrame(toote_info(x))
    date = pd.to_datetime('today').strftime('%Y-%m-%d')
    df['date'] = date
    df.replace('', np.nan, inplace=True)
    columns = ['date', 'name', 'rooms', 'price', 'pricesqm', 'numoffloor', 'geoloc' , 'builtyear', 'energymark', 'condition', 'link']
    df = df.reindex(columns=columns)
    df.drop_duplicates(subset=None, keep='first', inplace=True)
    '''Filter out data from dataframe'''
    columns = ['date', 'name', 'rooms', 'price', 'pricesqm', 'numoffloor', 'geoloc' , 'builtyear', 'energymark', 'condition', 'link']
    df = df.reindex(columns=columns)
    #df.drop_duplicates(subset=None, keep='first', inplace=True)
    '''How many missing values are in the dataframe'''
    nans = df.isnull().sum().sum()
    nan_rows = df.isnull().sum()
    if nans > 0:
        print('Sum of missing values', nans, '\n',
              'missing values per category \n', nan_rows)
    else:
        print('No missing values in dataframe')
    return df


if __name__ == '__main__':
    algandmed('links.txt')



 Pages that will be scraped: 

 ['https://www.kv.ee/?act=search.simple&page=1']
50
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions occured
No exceptions 

https://www.kv.ee/muua-uued-25-toalised-korterid-kadrioru-asumis-kor-3210131.html?nr=25&search_key=c969965abaf64f3b5db75e0e0c896ab1
['3', '2', '3', '3', '3', '3', '2', '3', '1', '3', '3', '2', '2', '4', '2', '2', '3', '2', '3', '3', '4', '350 m²', '3', '2', '3']
['1959', '1974', '2021', '2020', '2005', '1983', '2021', '2019', '2019', '2020', '2021', '2019', '2021', '2020', '2020', '1964', '2019', '2019', '2019', '2019', '2019', '2020']
[]
[]
https://www.kv.ee/raadimoisa-kodu-on-eesti-tunnustatud-arhitektiburo-3154250.html?nr=26&search_key=c969965abaf64f3b5db75e0e0c896ab1
['3', '2', '3', '3', '3', '3', '2', '3', '1', '3', '3', '2', '2', '4', '2', '2', '3', '2', '3', '3', '4', '350 m²', '3', '2', '3', '4']
['1959', '1974', '2021', '2020', '2005', '1983', '2021', '2019', '2019', '2020', '2021', '2019', '2021', '2020', '2020', '1964', '2019', '2019', '2019', '2019', '2019', '2020', '2020']
[]
[]
https://www.kv.ee/kadrioru-veerele-vaid-monesaja-meetri-kaugusele-ku-3182012.html?nr=27&search_

['1959', '1974', '2021', '2020', '2005', '1983', '2021', '2019', '2019', '2020', '2021', '2019', '2021', '2020', '2020', '1964', '2019', '2019', '2019', '2019', '2019', '2020', '2020', '2019', '2019', '2019', '1976', '2019', '2020', '2019', '2020', '2020', '2020', '2007']
[]
[]
https://www.kv.ee/rand-linnavillade-viimased-vabad-korterid-on-sisse-3017346.html?nr=41&search_key=c969965abaf64f3b5db75e0e0c896ab1
['3', '2', '3', '3', '3', '3', '2', '3', '1', '3', '3', '2', '2', '4', '2', '2', '3', '2', '3', '3', '4', '350 m²', '3', '2', '3', '4', '2', '3', '2', '4', '2', '4', '3', '3', '3', '4', '2', '1', '2', '2', '3']
['1959', '1974', '2021', '2020', '2005', '1983', '2021', '2019', '2019', '2020', '2021', '2019', '2021', '2020', '2020', '1964', '2019', '2019', '2019', '2019', '2019', '2020', '2020', '2019', '2019', '2019', '1976', '2019', '2020', '2019', '2020', '2020', '2020', '2007', '2019']
[]
[]
https://www.kv.ee/taiesti-uus-kodu-kaasaegses-elurajoonis-ootab-sind-3219165.html?nr=42&sea